In [1]:
const async = require('async');
const fs = require('fs');

# parallel

이전 함수 기다리지X, 함수 모음을 병렬로 실행

함수 중 하나라도 Error -> 메인 콜백 오류 값 호출

작업 정상 완료 -> 결과는 배열로 최종 콜백에 전달

자바스크립트는 단일 스레드이므로 코드는 직렬로, I/O 작업은 병렬로

작업이 실패했을 때 다른 작업을 계속 진행하려면 `reflect` 사용

In [4]:
// 콜백 사용
async.parallel([
    function(callback) {
        setTimeout(function() {
            callback(null, 'one');
        }, 200);
    },
    function(callback) {
        setTimeout(function() {
            callback(null, 'two');
        }, 100);
    }
], function(err, results) {
    console.log(results);
    // results는 ['one','two']와 같습니다.
    // 두 번째 함수의 타임아웃이 더 짧았지만 결과는 순서대로 나옵니다.
});

// 배열 대신 객체를 사용하는 예시
async.parallel({
    one: function(callback) {
        setTimeout(function() {
            callback(null, 1);
        }, 200);
    },
    two: function(callback) {
        setTimeout(function() {
            callback(null, 2);
        }, 100);
    }
}, function(err, results) {
    console.log(results);
    // results는 { one: 1, two: 2 }와 같습니다.
});

// 프로미스 사용
async.parallel([
    function(callback) {
        setTimeout(function() {
            callback(null, 'one');
        }, 200);
    },
    function(callback) {
        setTimeout(function() {
            callback(null, 'two');
        }, 100);
    }
]).then(results => {
    console.log(results);
    // results는 ['one','two']와 같습니다.
    // 두 번째 함수의 타임아웃이 더 짧았지만 결과는 순서대로 나옵니다.
}).catch(err => {
    console.log(err);
});

// 배열 대신 객체를 사용하는 예시
async.parallel({
    one: function(callback) {
        setTimeout(function() {
            callback(null, 1);
        }, 200);
    },
    two: function(callback) {
        setTimeout(function() {
            callback(null, 2);
        }, 100);
    }
}).then(results => {
    console.log(results);
    // results는 { one: 1, two: 2 }와 같습니다.
}).catch(err => {
    console.log(err);
});

// async/await 사용
async () => {
    try {
        let results = await async.parallel([
            function(callback) {
                setTimeout(function() {
                    callback(null, 'one');
                }, 200);
            },
            function(callback) {
                setTimeout(function() {
                    callback(null, 'two');
                }, 100);
            }
        ]);
        console.log(results);
        // results는 ['one','two']와 같습니다.
        // 두 번째 함수의 타임아웃이 더 짧았지만 결과는 순서대로 나옵니다.
    }
    catch (err) {
        console.log(err);
    }
}

// 배열 대신 객체를 사용하는 예시
async () => {
    try {
        let results = await async.parallel({
            one: function(callback) {
                setTimeout(function() {
                    callback(null, 1);
                }, 200);
            },
           two: function(callback) {
                setTimeout(function() {
                    callback(null, 2);
                }, 100);
           }
        });
        console.log(results);
        // results is equal to: { one: 1, two: 2 }
    }
    catch (err) {
        console.log(err);
    }
}

[AsyncFunction (anonymous)]

[ 'one', 'two' ]
{ two: 2, one: 1 }
[ 'one', 'two' ]
{ two: 2, one: 1 }


# parallelList

`parallel`과 동일하지만 한 번에 비동기 작업의 최댓값 제한이 있음

# retry

Error가 나왔을 때 성공적인 응답을 얻으려 재시도

작업이 성공하면 성공한 작업 결과 콜백 전달

모든 시도 실패하면 콜백은 오류와 최종 시도의 결과 전달

파라미터: opts, task, callback

- opts : number인 경우 재시도 횟수 의미
- times : 포기하기 전 시도 횟수 (기본 times는 5)
- interval : 재시도 밀리초 간격 (기본 interval은 0)\
- errorFilter : 잘못된 결과에 대해 호출되는 동기 함수 (true를 반환하면 재시도 반복, false를 반환하면 재시도 중단)

In [5]:
// `retry` 함수는 아래와 같이 콜백을 전달하여 독립적인 제어 흐름으로 사용할 수 있습니다.

// apiMethod를 3번 호출 시도
async.retry(3, apiMethod, function(err, result) {
    // 결과를 처리하는 코드
});

// apiMethod를 3번 호출 시도, 각 시도 사이에 200ms 대기
async.retry({times: 3, interval: 200}, apiMethod, function(err, result) {
    // 결과를 처리하는 코드
});

// apiMethod를 10번 호출 시도, 지수 백오프 사용
// (즉, 100, 200, 400, 800, 1600, ... 밀리초 간격)
async.retry({
  times: 10,
  interval: function(retryCount) {
    return 50 * Math.pow(2, retryCount);
  }
}, apiMethod, function(err, result) {
    // 결과를 처리하는 코드
});

// 기본값으로 apiMethod를 5번 호출 시도, 각 시도 사이에 지연 없음
async.retry(apiMethod, function(err, result) {
    // 결과를 처리하는 코드
});

// 에러 조건을 만족할 때만 apiMethod를 호출 시도, 다른 에러는
// 재시도 제어 흐름을 중단하고 최종 콜백으로 반환
async.retry({
  errorFilter: function(err) {
    return err.message === 'Temporary error'; // 특정 에러일 때만 재시도
  }
}, apiMethod, function(err, result) {
    // 결과를 처리하는 코드
});

// 다른 제어 흐름 함수 내에서 신뢰할 수 없는 개별 메서드를 재시도하려면
// `retryable` 래퍼를 사용:
async.auto({
    users: api.getUsers.bind(api),
    payments: async.retryable(3, api.getPayments.bind(api))
}, function(err, results) {
    // 결과를 처리하는 코드
});


ReferenceError: apiMethod is not defined

# times

반복함수를 n번 호출하고 map에서 사용하는 것과 동일한 방식으로 결과 누적

In [6]:
// 복잡한 비동기 팩토리라고 가정
var createUser = function(id, callback) {
    callback(null, {
        id: 'user' + id
    });
};

// 사용자 5명 생성
async.times(5, function(n, next) {
    createUser(n, function(err, user) {
        next(err, user);
    });
}, function(err, users) {
    // 이제 5명의 사용자가 있어야 합니다.
    console.log(users);
    // 출력 예시: [{ id: 'user0' }, { id: 'user1' }, { id: 'user2' }, { id: 'user3' }, { id: 'user4' }]
});


[
  { id: 'user0' },
  { id: 'user1' },
  { id: 'user2' },
  { id: 'user3' },
  { id: 'user4' }
]


# waterfall

함수 배열 순서대로(직렬로) 진행. 로직 구현 시 가독성이 좋아 자주 사용

비동기를 동기로 사용하게 하는 것

각 작업 결과를 배열의 다음 작업으로 전달

but 작업 중 하나라도 Error 나오면 다음 함수 실행 X

In [10]:
async.waterfall([
    function(callback) {
        callback(null, 'one', 'two');
    },
    function(arg1, arg2, callback) {
        // arg1은 이제 'one'이고 arg2는 이제 'two'입니다.
        callback(null, 'three');
    },
    function(arg1, callback) {
        // arg1은 이제 'three'입니다.
        callback(null, 'done');
    }
], function (err, result) {
    // result는 이제 'done'입니다.
    console.log(result);
});

// 또는, 이름이 있는 함수로:
async.waterfall([
    myFirstFunction,
    mySecondFunction,
    myLastFunction,
], function (err, result) {
    // result는 이제 'done'입니다.
    console.log(result);
});

function myFirstFunction(callback) {
    callback(null, 'one', 'two');
}
function mySecondFunction(arg1, arg2, callback) {
    // arg1은 이제 'one'이고 arg2는 이제 'two'입니다.
    callback(null, 'three');
}
function myLastFunction(arg1, callback) {
    // arg1은 이제 'three'입니다.
    callback(null, 'done');
}


done
done
